In [1]:
data_var = '2024-01-09'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9880,2024-01-09,Islândia Premier League Feminina,16:15,Grindavik F,Hafnarfjordur F,1.31,3.25,145.5,1.83,1.83,-8.5,1.95,1.72,fPlPvSBI,0.763359,0.307692,0.546448,0.546448,0.071051,121.812,25.120989,0.206228,1.8,1.643168,0.912871,130.41,1.514,0.425711,0.281183,44.0,229.578,197.257555,0.859218,1.8,1.643168,0.912871,536.90,3.644,3.525852,0.967577,7.0,63,59,2.07,9.10,0.000,239.560,0.601661,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,-2.58,-0.516,-4.360465,0.0,0.0,0.0
9881,2024-01-09,Israel Liga Leumit,13:30,Hapoel Bnei Kfar Kasem,Elitzur Ashkelon,1.65,2.15,167.5,1.85,1.85,-4.5,2.08,1.63,QBF8S9Tq,0.606061,0.465116,0.540541,0.540541,0.071177,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,101,102,2.15,1.30,0.000,0.000,0.186081,0.000000,0.171535,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
9882,2024-01-09,Israel Liga Leumit,14:00,Elitzur Shomron,Maccabi Rishon,1.56,2.32,166.5,1.85,1.85,-3.5,1.85,1.85,zscakXzc,0.641026,0.431034,0.540541,0.540541,0.072060,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,88,68,1.99,1.74,0.000,0.000,0.277011,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
9883,2024-01-09,Lituânia Nkl,13:00,Suduva-Mantinga,Vytis VDU,1.11,5.77,165.5,1.86,1.84,-13.5,2.00,1.72,WIZceXdI,0.900901,0.173310,0.537634,0.543478,0.074211,110.530,16.253526,0.147051,2.4,1.341641,0.559017,99.76,1.278,0.402765,0.315153,87.0,123.094,27.460784,0.223088,1.8,1.643168,0.912871,149.33,1.478,0.472779,0.319878,55.0,86,109,1.16,1.37,123.348,124.998,0.957883,0.007644,0.106446,1.90,0.380,0.289474,0.000000,0.0,0.000000,2.49,0.498,9.578313,0.0,0.0,0.0
9884,2024-01-09,Israel Liga Leumit,14:30,Safed,Maccabi Maale Adumim,1.65,2.15,162.5,1.85,1.85,-4.5,2.08,1.63,rmADRTrj,0.606061,0.465116,0.540541,0.540541,0.071177,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,78,71,3.02,3.61,0.000,0.000,0.186081,0.000000,0.171535,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,2024-01-09,Eua Ncaa,21:00,Alabama,South Carolina,1.14,5.60,155.5,1.91,1.91,-13.5,2.10,1.67,fi7xo211,0.877193,0.178571,0.523560,0.523560,0.055764,170.408,89.041290,0.522518,1.8,1.643168,0.912871,84.24,2.032,1.286009,0.632879,76.0,97.524,21.175639,0.217133,3.0,0.000000,0.000000,129.88,1.336,0.375073,0.280744,81.0,78,68,1.08,1.91,101.032,0.000,0.935815,0.000000,0.161303,-0.92,-0.184,-0.760870,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
9968,2024-01-09,Eua Ncaa,20:30,Georgetown,Seton Hall,2.87,1.41,141.5,1.91,1.91,4.5,2.10,1.67,KOB9sMRr,0.348432,0.709220,0.523560,0.523560,0.057652,319.392,180.266694,0.564406,1.2,1.643168,1.369306,95.88,5.434,3.736252,0.687569,-50.0,239.006,146.528573,0.613075,2.4,1.341641,0.559017,213.72,3.302,1.865883

## Apostas do dia

### Back Home V3 (QCut)

In [12]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9911,21:00,Eua Ncaa,Buffalo,Miami (Ohio),1.77,Back Home
9927,07:00,Coreia Do Sul Kbl,KCC Egis,Mobis Phoebus,1.54,Back Home
